<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/misc_examples/Splitting_a_keras_model_into_two_halves_using_intermediate_layer_as_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import keras

Using TensorFlow backend.


In [3]:
!wget https://zenodo.org/record/2648107/files/model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5?download=1 -O themodel.h5

--2020-07-27 04:31:28--  https://zenodo.org/record/2648107/files/model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60157216 (57M) [application/octet-stream]
Saving to: ‘themodel.h5’

themodel.h5         100%[===================>]  57.37M  13.6MB/s    in 4.2s    

2020-07-27 04:31:33 (13.6 MB/s) - ‘themodel.h5’ saved [60157216/60157216]



In [4]:
model = keras.models.load_model("themodel.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)       

In [6]:
model.layers[1].input_shape

(None, 32, 32, 64)

In [7]:
def copy_layer(layer_to_copy, new_config_extra_kwargs={}):
  new_config = layer_to_copy.get_config()
  new_config.update(new_config_extra_kwargs)
  new_layer = layer_to_copy.__class__.from_config(new_config)
  new_layer.build(layer_to_copy.input_shape)
  new_layer.set_weights(layer_to_copy.get_weights())
  return new_layer

def create_subset_model(model, layer_to_start_at, layer_to_end_at):
  sequential_model = keras.models.Sequential()
  layers_to_add = model.layers[layer_to_start_at:layer_to_end_at]

  new_first_layer = copy_layer(
      layer_to_copy=layers_to_add[0],
      new_config_extra_kwargs={
          'batch_input_shape': layers_to_add[0].input_shape
      })
  sequential_model.add(new_first_layer)
  for layer in layers_to_add[1:]:
    print("layer", layer)
    sequential_model.add(copy_layer(layer_to_copy=layer))

  return sequential_model

def split_model(model, layer_to_split_at):
  first_half = create_subset_model(model=model,
                  layer_to_start_at=0,
                  layer_to_end_at=layer_to_split_at)
  second_half = create_subset_model(model=model,
                  layer_to_start_at=layer_to_split_at,
                  layer_to_end_at=len(model.layers))
  return first_half, second_half

In [8]:
first_half_model, second_half_model = split_model(model=model,
                                                  layer_to_split_at=-9)

layer <keras.layers.core.Activation object at 0x7ff0a949bd68>
layer <keras.layers.normalization.BatchNormalization object at 0x7ff0a949b9b0>
layer <keras.layers.core.Dropout object at 0x7ff0a98f4e80>
layer <keras.layers.convolutional.Conv2D object at 0x7ff0a949be10>
layer <keras.layers.core.Activation object at 0x7ff0a8c66f28>
layer <keras.layers.normalization.BatchNormalization object at 0x7ff0a276e898>
layer <keras.layers.pooling.MaxPooling2D object at 0x7ff0a27676a0>
layer <keras.layers.convolutional.Conv2D object at 0x7ff0a276e550>
layer <keras.layers.core.Activation object at 0x7ff0a270e8d0>
layer <keras.layers.normalization.BatchNormalization object at 0x7ff0a26b6b00>
layer <keras.layers.core.Dropout object at 0x7ff0a2724940>
layer <keras.layers.convolutional.Conv2D object at 0x7ff0a26b67b8>
layer <keras.layers.core.Activation object at 0x7ff0a26cafd0>
layer <keras.layers.normalization.BatchNormalization object at 0x7ff0a2683860>
layer <keras.layers.pooling.MaxPooling2D object at

In [9]:
model.input_shape

(None, 32, 32, 3)

In [10]:
import numpy as np
an_input = np.random.RandomState(1234).random((1,32,32,3))
print(model.predict(an_input))
print(second_half_model.predict(first_half_model.predict(an_input)))


[[4.1146105e-04 3.6833601e-06 8.3194032e-02 4.8545543e-02 2.1070623e-01
  1.1816010e-03 6.5564293e-01 6.7880363e-05 1.1103776e-04 1.3561430e-04]]
[[4.1146105e-04 3.6833601e-06 8.3194032e-02 4.8545543e-02 2.1070623e-01
  1.1816010e-03 6.5564293e-01 6.7880363e-05 1.1103776e-04 1.3561430e-04]]
